In [6]:
import pandas as pd

purchase_data = pd.read_csv("Resources/purchase_data.csv")
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [11]:
count_by_sn = len(purchase_data["SN"].unique())
total_player_df = pd.DataFrame({"Total Players":[count_by_sn]})
total_player_df

,Total Players
0,576


In [312]:
num_unique_items = len(purchase_data["Item Name"].unique())
avg_purchase_price = purchase_data["Price"].mean()
num_purchases = len(purchase_data["Purchase ID"])
revenue = purchase_data["Price"].sum()

format_dict = {"Average Purchase Price":"${0:,.2f}", "Total Revenue":"${0:,.2f}", "Total Purchase Value":"${0:,.2f}", 
               "Average Purchase Total per Person":"${0:,.2f}", "Item Price":"${0:,.2f}"}

purchasing_analysis =pd.DataFrame({"Number of Unique Items":[num_unique_items],
                                   "Average Purchase Price":avg_purchase_price,
                                   "Total Number of Purchases":[num_purchases],
                                   "Total Revenue":revenue})
purchasing_analysis.style.format(format_dict)

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [105]:
sn_gender_df = purchase_data.set_index("Gender")

male_df = sn_gender_df.loc[["Male"], :]
female_df = sn_gender_df.loc[["Female"], :]
other_df = sn_gender_df.loc[["Other / Non-Disclosed"], :]

unique_male_count = len(male_df["SN"].unique())
unique_female_coun = len(female_df["SN"].unique())
unique_other_count = len(other_df["SN"].unique())

percent_male = unique_male_count/count_by_sn
percent_female = unique_female_count/count_by_sn
percent_other = unique_other_count/count_by_sn

gender_demographics = pd.DataFrame({"Gender":["Male", "Female", "Other / Non-Disclosed"],
                                    "Total Count":[unique_male_count, unique_female_count, unique_other_count],
                                    "Percentage of Players":[percent_male, percent_female, percent_other]})
gender_demographics.style.format({"Percentage of Players":"{:.2%}"}).hide_index()

Gender,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [146]:
male_purchase_count = len(male_df["Purchase ID"])
female_purchase_count = len(female_df["Purchase ID"])
other_purchase_count = len(other_df["Purchase ID"])

male_avg_purchase_price = male_df["Price"].mean()
female_avg_purchase_price = female_df["Price"].mean()
other_avg_purchase_price = other_df["Price"].mean()

male_purchase_value = male_df["Price"].sum()
female_purchase_value = female_df["Price"].sum()
other_purchase_value = other_df["Price"].sum()

male_avg_sn = male_purchase_value/unique_male_count
female_avg_sn = female_purchase_value/unique_female_count
other_avg_sn = other_purchase_value/unique_other_count

purchase_analysis_gender = pd.DataFrame({"Gender":["Male", "Female", "Other / Non-Disclosed"],
                                        "Purchase Count":[male_purchase_count, female_purchase_count, other_purchase_count],
                                        "Average Purchase Price":[male_avg_purchase_price, female_avg_purchase_price, other_avg_purchase_price],
                                        "Total Purchase Value":[male_purchase_value, female_purchase_value, other_purchase_value],
                                        "Average Purchase Total per Person":[male_avg_sn, female_avg_sn, other_avg_sn]})
purchase_analysis_gender.style.format(format_dict).hide_index()

Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [156]:
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 45]
age_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
purchase_data["Age Grouping"] = pd.cut(purchase_data["Age"], age_bins, labels=age_names, include_lowest=True)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Grouping
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [217]:
age_grouped = purchase_data.groupby(["Age Grouping"])
age_group_df = age_grouped["SN"].nunique().to_frame(name="Player Count")
age_group_df["Percentage of Players"] = age_group_df["Player Count"]/count_by_sn
age_group_df.style.format({"Percentage of Players":"{:.2%}"})

,Player Count,Percentage of Players
Age Grouping,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [266]:
age_group_total_players = age_grouped["Purchase ID"].count().to_frame(name="Purchase Count")
age_group_prices = age_grouped["Price"].sum().to_frame(name="Total Purchase Value")
age_group_demo = pd.merge(age_group_total_players, age_group_prices, on="Age Grouping")
age_group_demo["Average Purchase Price"] = age_group_demo["Total Purchase Value"]/age_group_demo["Purchase Count"]
age_group_demo["Average Purchase Total per Person"] = age_group_demo["Total Purchase Value"]/age_group_df["Player Count"]
age_group_demo = age_group_demo[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Average Purchase Total per Person"]]
age_group_demo.style.format(format_dict)

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Age Grouping,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [265]:
sn_grouped = purchase_data.groupby(["SN"])
sn_grouped_prices = sn_grouped["Price"].sum().to_frame(name="Total Purchase Value")
sn_grouped_purchase_count = sn_grouped["Purchase ID"].count().to_frame(name="Purchase Count")
sn_grouped_df = pd.merge(sn_grouped_prices, sn_grouped_purchase_count, on="SN")

top_spenders = sn_grouped_df.nlargest(n=5, columns=("Total Purchase Value"))

top_spenders["Average Purchase Price"] = top_spenders["Total Purchase Value"]/top_spenders["Purchase Count"]
top_spenders = top_spenders[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]
top_spenders.style.format(format_dict)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [313]:
item_grouped = purchase_data.groupby(["Item ID", "Item Name"])

item_grouped_df = pd.DataFrame(item_grouped[["Price"]].sum())
item_grouped_df["Purchase Count"] = item_grouped[["Purchase ID"]].count()
top_items = item_grouped_df.nlargest(n=5, columns=("Purchase Count"))
top_items = top_items.rename(columns={"Price":"Total Purchase Value", })
top_items["Item Price"] = top_items["Total Purchase Value"]/top_items["Purchase Count"]
top_items = top_items[["Purchase Count", "Item Price", "Total Purchase Value"]]
top_items.style.format(format_dict)


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
132,Persuasion,9,$3.22,$28.99


In [316]:
profit_items = item_grouped_df.nlargest(n=5, columns=("Price"))
profit_items = profit_items.rename(columns={"Price":"Total Purchase Value", })
profit_items["Item Price"] = profit_items["Total Purchase Value"]/profit_items["Purchase Count"]
profit_items = profit_items[["Purchase Count", "Item Price", "Total Purchase Value"]]
profit_items.style.format(format_dict)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
